In [1]:
from functions.satellites_ephemeris import SatelliteEphemeris

filename = 'functions/ephemerides/OneWeb_Sat_Data.xlsx'
ephemeris = SatelliteEphemeris(filename=filename)
ephemeris.load_data()
debris_data = ephemeris.get_debris()

Custom database loaded.


In [2]:
t1 = 8487.72551337956
id1 = '0'

t2 = 8492.72551337956
id2 = '1'

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

In [3]:
from functions.MRPLP_J2_analytic import MultiRevolutionPerturbedLambertSolver
from functions.expansion_perturbed_lambert import ExpansionPerturbedLambert
import numpy as np

rr1=cart1[0:3]
vv1=cart1[3:]
rr2=cart2[0:3]
vv2=cart2[3:]
vv1g = vv1

tof = ( t2 - t1 ) * 86400

mu = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

MRPLPsolver = MultiRevolutionPerturbedLambertSolver() # MRPLP solver
EXPpertLamb = ExpansionPerturbedLambert() # Expansion of perturbed Lambert

# set the parameters for the MRPLP solver
order = 7
parameters = MRPLPsolver.mrplp_J2_analytic_parameters(rr1, rr2, tof, vv1g, mu, Re, J2,
                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, scl=1.0e-3, itermax=200 )

# solve the MRPLP
output = MRPLPsolver.mrplp_J2_analytic(parameters)

# extract the solution
vv1Sol = output.vv1Sol
vv2Sol = output.vv2Sol
rr2DA = output.rr2DA

# compute the DV
dvv1 = vv1Sol - vv1
dvv2 = vv2 - vv2Sol
dv1 = np.linalg.norm( dvv1 )
dv2 = np.linalg.norm( dvv2 )
dvtot = dv1 + dv2

# print the output
print(f"-------------------------------------------------------")
print(f"                 OUTPUT SUMMARY")
print(f"-------------------------------------------------------")
print(f"Order of the expansion  : {order}")
print(f"Success                 : {output.success}")
print(f"Elapsed time            : {output.elapsed_time} seconds")
print(f"Final pos. error (norm) : {np.linalg.norm( output.rr2DA - rr2 )} m")
print(f"-------------------------------------------------------")
print(f"Delta_v1                : {dv1} m/s")
print(f"Delta_v2                : {dv2} m/s")
print(f"Delta_vtot              : {dvtot} m/s")
print(f"-------------------------------------------------------")


-------------------------------------------------------
                 OUTPUT SUMMARY
-------------------------------------------------------
Order of the expansion  : 7
Success                 : True
Elapsed time            : 0.29102206230163574 seconds
Final pos. error (norm) : 2.1105781332667353e-06 m
-------------------------------------------------------
Delta_v1                : 22.71146522768374 m/s
Delta_v2                : 21.014510169245128 m/s
Delta_vtot              : 43.72597539692887 m/s
-------------------------------------------------------


In [4]:
# propagate and obtain the STM
states, vecttof, STM = (
    MultiRevolutionPerturbedLambertSolver.propagatePerturbedKeplerSTM(
        rr1, vv1Sol, tof, 1000.0, parameters
    )
)

In [7]:
states

array([[ 6.81353729e+05, -4.00880032e+06,  6.39074168e+06,
         1.50859783e+03, -5.93272359e+03, -3.86342442e+03],
       [ 1.12491453e+06, -3.84404813e+06, -6.42934739e+06,
        -1.23854620e+03,  6.01504840e+03, -3.83159731e+03],
       [ 5.10429326e+05, -9.95782134e+05, -7.49662164e+06,
        -1.56561682e+03,  6.98313595e+03, -1.04057263e+03],
       ...,
       [ 9.98655985e+05, -3.55332191e+06, -6.61965943e+06,
        -1.18052626e+03,  6.20851831e+03, -3.52027012e+03],
       [ 4.19237879e+05, -6.48585055e+05, -7.54105520e+06,
        -1.46374216e+03,  7.04789706e+03, -6.85539194e+02],
       [-2.15442729e+05,  2.29773839e+06, -7.21763221e+06,
        -1.50343751e+03,  6.72625139e+03,  2.19828862e+03]])

In [9]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Extract x, y, z components
x = states[:, 0]
y = states[:, 1]
z = states[:, 2]

# Create interactive 3D plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(width=2),
    name='trajectory'
)])

# Add initial and final points
fig.add_trace(go.Scatter3d(
    x=[x[0]],
    y=[y[0]],
    z=[z[0]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[1]],
    y=[y[1]],
    z=[z[1]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[2]],
    y=[y[2]],
    z=[z[2]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[3]],
    y=[y[3]],
    z=[z[3]],
    mode='markers',
    marker=dict(size=5, color='green'),
    name='Initial Point'
))

fig.add_trace(go.Scatter3d(
    x=[x[-1]],
    y=[y[-1]],
    z=[z[-1]],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Final Point'
))

# Labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Show the plot
fig.show()

In [ ]:
# analytic computation of DV
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2
import numpy as np

mue = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

# Initialize parameters
parameters = AnalyticDVApproxJ2.approx_DV_parameters(
    rr1=cart1[0:3],
    vv1=cart1[3:],
    rr2=cart2[0:3],
    vv2=cart2[3:],
    t1=t1,
    t2=t2,
    mu=mue,
    rE=Re,
    J2=J2
)

delta_v = AnalyticDVApproxJ2.approx_DV(parameters)
delta_v